In [21]:
import os
os.environ['CUDA_LAZY_MODULE_LOADING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import re

In [22]:
df=pd.read_csv('train.csv')
df['deck'],df['num'],df['side']=df.Cabin.str.split('/').str
df['gggg'],df['pp']=df.PassengerId.str.split('_').str
df.gggg=df.gggg.astype(int)
df.pp=df.pp.astype(int)
df.Transported=df.Transported.astype(int)
df.CryoSleep = df.CryoSleep.fillna(-1)
df.CryoSleep = df.CryoSleep.astype(int)
df.CryoSleep = df.CryoSleep.replace(-1, np.nan)
df.VIP = df.VIP.fillna(-1)
df.VIP = df.VIP.astype(int)
df.VIP = df.VIP.replace(-1, np.nan)
df.num = df.num.fillna(-1)
df.num = df.num.astype(int)
df.num = df.num.replace(-1, np.nan)
df=df.drop('PassengerId',axis=1)
df=df.drop('Cabin',axis=1)
df.side = df.side.replace({'P': 0, 'S': 1}).fillna(np.nan)
df.deck = df.deck.replace({'B': 0, 'F': 1,'A': 2, 'G': 3,'E': 4, 'D': 5,'C': 6, 'T': 7}).fillna(np.nan)
df.Destination = df.Destination.replace({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1,'55 Cancri e': 2}).fillna(np.nan)
df.HomePlanet = df.HomePlanet.replace({'Europa': 0, 'Earth': 1,'Mars': 2}).fillna(np.nan)
df=df.drop('side',axis=1)
df=df.drop('Name',axis=1)
df=df.drop('Age',axis=1)
df=df.drop('pp',axis=1)

#df.side=df.side.astype(int)
#df.num=df.num.astype(int)
#df.deck=df.deck.astype(int)

/tmp/ipykernel_1753/3422561030.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['deck'],df['num'],df['side']=df.Cabin.str.split('/').str
/tmp/ipykernel_1753/3422561030.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['gggg'],df['pp']=df.PassengerId.str.split('_').str


In [23]:
df

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,gggg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,109.0,9.0,25.0,549.0,44.0,1,1.0,0.0,2
2,0.0,0.0,0.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,2.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,2.0,0.0,3
4,1.0,0.0,0.0,0.0,303.0,70.0,151.0,565.0,2.0,1,1.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0.0,2.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,2.0,98.0,9276
8689,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,1499.0,9278
8690,1.0,0.0,0.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,3.0,1500.0,9279
8691,0.0,0.0,2.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,4.0,608.0,9280


In [24]:
test=pd.read_csv('test.csv')
test['deck'],test['num'],test['side']=test.Cabin.str.split('/').str
test['gggg'],test['pp']=test.PassengerId.str.split('_').str
test.gggg=test.gggg.astype(int)
test.pp=test.pp.astype(int)
test.CryoSleep = test.CryoSleep.fillna(-1)
test.CryoSleep = test.CryoSleep.astype(int)
test.CryoSleep = test.CryoSleep.replace(-1, np.nan)
test.VIP = test.VIP.fillna(-1)
test.VIP = test.VIP.astype(int)
test.VIP = test.VIP.replace(-1, np.nan)
test.num = test.num.fillna(-1)
test.num = test.num.astype(int)
test.num = test.num.replace(-1, np.nan)
pas=test.pop('PassengerId')
test=test.drop('Cabin',axis=1)
test.side = test.side.replace({'P': 0, 'S': 1}).fillna(np.nan)
test.deck = test.deck.replace({'B': 0, 'F': 1,'A': 2, 'G': 3,'E': 4, 'D': 5,'C': 6, 'T': 7}).fillna(np.nan)
test.Destination = test.Destination.replace({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1,'55 Cancri e': 2}).fillna(np.nan)
test.HomePlanet = test.HomePlanet.replace({'Europa': 0, 'Earth': 1,'Mars': 2}).fillna(np.nan)
test=test.drop('side',axis=1)
test=test.drop('Name',axis=1)
test=test.drop('Age',axis=1)
test=test.drop('pp',axis=1)
#test.side=test.side.astype(int)
#test.num=test.num.astype(int)
#test.deck=test.deck.astype(int)

/tmp/ipykernel_1753/1503605996.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  test['deck'],test['num'],test['side']=test.Cabin.str.split('/').str
/tmp/ipykernel_1753/1503605996.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  test['gggg'],test['pp']=test.PassengerId.str.split('_').str


In [25]:
df.corr()

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,gggg
HomePlanet,1.000000,-0.042643,-0.251407,-0.069965,0.194054,-0.314618,0.082374,-0.229258,-0.253395,-0.103474,-0.169217,0.357507,-0.008609
CryoSleep,-0.042643,1.000000,0.096722,-0.081402,-0.252396,-0.211510,-0.212514,-0.203991,-0.198857,0.468645,0.037388,-0.037560,-0.005879
Destination,-0.251407,0.096722,1.000000,0.044722,-0.047685,0.112180,-0.025433,0.058329,0.073707,0.109806,0.037490,-0.105492,0.003659
VIP,-0.069965,-0.081402,0.044722,1.000000,0.058785,0.129799,0.018295,0.061059,0.125974,-0.037650,0.033734,-0.099416,0.014018
RoomService,0.194054,-0.252396,-0.047685,0.058785,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611,0.029319,-0.008986,-0.000346
FoodCourt,-0.314618,-0.211510,0.112180,0.129799,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566,0.109611,-0.183429,-0.009552
ShoppingMall,0.082374,-0.212514,-0.025433,0.018295,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141,-0.012830,0.001727,0.017397
Spa,-0.229258,-0.203991,0.058329,0.061059,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131,0.075083,-0.131342,-0.005211
VRDeck,-0.253395,-0.198857,0.073707,0.125974,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075,0.079417,-0.139080,0.015432
Transported,-0.103474,0.468645,0.109806,-0.037650,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000,0.001737,-0.045097,0.021491


In [26]:
y=df.pop('Transported')

In [27]:
selected_columns = ['HomePlanet','Destination', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','num','VIP','deck','CryoSleep','gggg']

In [28]:
from sklearn.impute import  KNNImputer,SimpleImputer
knn_imputer = KNNImputer(n_neighbors=4)
imputed_data = knn_imputer.fit_transform(test[selected_columns])
test[selected_columns] = imputed_data

In [29]:
from sklearn.impute import  KNNImputer,SimpleImputer
knn_imputer = KNNImputer(n_neighbors=4)
imputed_data = knn_imputer.fit_transform(df[selected_columns])
df[selected_columns] = imputed_data

In [30]:
# df=df.drop('gggg',axis=1)
# df=df.drop('num',axis=1)
# df=df.drop('ShoppingMall',axis=1)
# df=df.drop('FoodCourt',axis=1)
# test=test.drop('gggg',axis=1)
# test=test.drop('num',axis=1)
# test=test.drop('ShoppingMall',axis=1)
# test=test.drop('FoodCourt',axis=1)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
mm=MinMaxScaler()
df=mm.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [4277, 8693]

In [42]:
from sklearn.metrics import accuracy_score

import numpy as np
classi=RandomForestClassifier(max_depth=11)
classi.fit(X_train,y_train)
with np.warnings.catch_warnings():
    np.warnings.filterwarnings('ignore', category=FutureWarning)

    # Your code for making predictions and calculating accuracy

    y_test_pred = classi.predict(X_test)
    y_train_pred = classi.predict(X_train)

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    
    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    


Train Accuracy: 0.8851
Test Accuracy: 0.7839


In [33]:
ans=pd.DataFrame({'PassengerId': pas,'Transported':classi.predict(test)==1}).set_index('PassengerId')

/home/butt_sahab/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [34]:
ans

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,False
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [35]:
ans.to_csv('answer.csv')